In [ ]:
def plot_comparison(cpu: TimingDataset, cuda: TimingDataset | None = None) -> None:
    cpu_df = cpu.to_dataframe().copy()
    cpu_df.rename(columns={
        cpu.header[1]: "MatrixTime_CPU",
        cpu.header[2]: "SolveTime_CPU",
        cpu.header[3]: "TotalTime_CPU",
    }, inplace=True)

    if cuda is not None:
        cuda_df = cuda.to_dataframe().copy()
        cuda_df.rename(columns={
            cuda.header[1]: "MatrixTime_CUDA",
            cuda.header[2]: "SolveTime_CUDA",
            cuda.header[3]: "TotalTime_CUDA",
        }, inplace=True)
        merged = cpu_df.merge(cuda_df, on="N", how="inner")
    else:
        merged = cpu_df

    display(merged)

    plt.style.use("seaborn-v0_8")
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(merged["N"], merged["MatrixTime_CPU"], "o-", label="CPU матрица")
    if "MatrixTime_CUDA" in merged:
        ax.plot(merged["N"], merged["MatrixTime_CUDA"], "s-", label="CUDA матрица")
    ax.set_xlabel("N")
    ax.set_ylabel("Время (мс)")
    ax.set_title("Построение матрицы")
    ax.grid(True, alpha=0.3)
    ax.legend()
    fig.tight_layout()
    fig.savefig(PLOTS_DIR / "matrix_compare.png", dpi=300)

    fig2, ax2 = plt.subplots(figsize=(10, 6))
    ax2.plot(merged["N"], merged["TotalTime_CPU"], "o-", label="CPU общее")
    if "TotalTime_CUDA" in merged:
        ax2.plot(merged["N"], merged["TotalTime_CUDA"], "s-", label="CUDA общее")
    ax2.set_xlabel("N")
    ax2.set_ylabel("Время (мс)")
    ax2.set_title("Полное время")
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    fig2.tight_layout()
    fig2.savefig(PLOTS_DIR / "total_compare.png", dpi=300)

    if "MatrixTime_CUDA" in merged:
        fig3, ax3 = plt.subplots(figsize=(10, 6))
        speed_matrix = merged["MatrixTime_CPU"] / merged["MatrixTime_CUDA"]
        speed_total = merged["TotalTime_CPU"] / merged["TotalTime_CUDA"]
        ax3.bar(merged["N"] - 1, speed_matrix, width=2, label="Матрица")
        ax3.bar(merged["N"] + 1, speed_total, width=2, label="Общее")
        ax3.set_xlabel("N")
        ax3.set_ylabel("Ускорение CPU/CUDA")
        ax3.set_title("GPU speedup")
        ax3.grid(True, axis="y", alpha=0.3)
        ax3.legend()
        fig3.tight_layout()
        
        fig3.savefig(PLOTS_DIR / "speedup.png", dpi=300)

        if "MatrixTime_CUDA" in merged:
            # New Plot: GPU Distribution (Percentage)
            fig4, ax4 = plt.subplots(figsize=(10, 6))
            
            # Calculate percentages for GPU
            gpu_total = merged["TotalTime_CUDA"]
            gpu_total = gpu_total.replace(0, 1e-10) # Avoid division by zero
            
            pct_matrix_gpu = (merged["MatrixTime_CUDA"] / gpu_total) * 100
            pct_solve_gpu = (merged["SolveTime_CUDA"] / gpu_total) * 100
            
            if len(merged) > 1:
                width = min(5, (merged["N"].iloc[1] - merged["N"].iloc[0]) / 3)
            else:
                width = 5
                
            N_vals = merged["N"]
            
            ax4.bar(N_vals - width/2, pct_matrix_gpu, width, label='Построение матрицы (GPU)', color='#2E86AB', alpha=0.8)
            ax4.bar(N_vals + width/2, pct_solve_gpu, width, label='Решение СЛАУ (GPU)', color='#A23B72', alpha=0.8)
            
            ax4.set_xlabel('Параметр усечения N', fontsize=12, fontweight='bold')
            ax4.set_ylabel('Доля времени (%)', fontsize=12, fontweight='bold')
            ax4.set_title('Распределение времени выполнения на GPU', fontsize=14, fontweight='bold')
            ax4.legend(loc='upper right')
            ax4.grid(True, axis='y', alpha=0.3)
            ax4.set_ylim(0, 115)
            
            for i, n in enumerate(N_vals):
                ax4.text(n - width/2, pct_matrix_gpu.iloc[i] + 2, f'{pct_matrix_gpu.iloc[i]:.1f}%', ha='center', va='bottom', fontsize=8, rotation=90)
                ax4.text(n + width/2, pct_solve_gpu.iloc[i] + 2, f'{pct_solve_gpu.iloc[i]:.1f}%', ha='center', va='bottom', fontsize=8, rotation=90)
                
            fig4.tight_layout()
            fig4.savefig(PLOTS_DIR / "gpu_distribution.png", dpi=300)

    print(f"Графики сохранены в {PLOTS_DIR}")
